In [1]:
import pycisTopic
import glob
import os
import pybiomart as pbm
import pandas as pd
import pickle
from pycisTopic.qc import *
from IPython.display import Image, display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import multiprocess as mp  # for kde multithreading calculation
from multiprocess import Pool

%matplotlib inline
%load_ext lab_black

# Download annotation

In [2]:
!pwd

/lustre1/project/stg_00090/scatac_benchmark/fixedcells_3_cistopic_consensus


In [4]:
wdir = "/lustre1/project/stg_00090/scatac_benchmark/fixedcells_3_cistopic_consensus"
os.chdir(wdir)

In [5]:
genome = "hg38"

pbm_genome_name_dict = {
    "hg38": "hsapiens_gene_ensembl",
    "hg37": "hsapiens_gene_ensembl",
    "mm10": "mmusculus_gene_ensembl",
    "dm6": "dmelanogaster_gene_ensembl",
}

pbm_host_dict = {
    "hg38": "http://www.ensembl.org",
    "hg37": "http://grch37.ensembl.org/",
    "mm10": "http://nov2020.archive.ensembl.org/",
    "dm6": "http://www.ensembl.org",
}

if os.path.exists(f"annotation.tsv"):
    print(f"Loading cached genome annotation...")
    annotation = pd.read_csv("annotation.tsv", sep="\t", header=0, index_col=0)
else:
    dataset = pbm.Dataset(name=pbm_genome_name_dict[genome], host=pbm_host_dict[genome])

    annotation = dataset.query(
        attributes=[
            "chromosome_name",
            "transcription_start_site",
            "strand",
            "external_gene_name",
            "transcript_biotype",
        ]
    )
    filter = annotation["Chromosome/scaffold name"].str.contains("CHR|GL|JH|MT")
    annotation = annotation[~filter]
    annotation["Chromosome/scaffold name"] = annotation[
        "Chromosome/scaffold name"
    ].str.replace(r"(\b\S)", r"chr\1")
    annotation.columns = ["Chromosome", "Start", "Strand", "Gene", "Transcript_type"]
    annottation = annotation[annotation.Transcript_type == "protein_coding"]
    annotation.to_csv("annotation.tsv", sep="\t")

Loading cached genome annotation...


In [6]:
annotation

,Chromosome,Start,Strand,Gene,Transcript_type
57,chrKI270711chr.chr1,24650,-1,NaN,protein_coding
58,chrKI270711chr.chr1,29626,-1,NaN,protein_coding
628,chrKI270734chr.chr1,161852,-1,NaN,protein_coding
629,chrKI270734chr.chr1,161750,-1,NaN,protein_coding
713,chrKI270734chr.chr1,131494,1,NaN,protein_coding
...,...,...,...,...,...
242597,chr1,154257071,1,UBAP2L,protein_coding
242604,chr1,154272589,1,HAX1,protein_coding
242605,chr1,154272609,1,HAX1,protein_coding
242607,chr1,154272629,1,HAX1,protein_coding


In [9]:
annotation

,Chromosome,Start,Strand,Gene,Transcript_type,End
57,chrKI270711chr.chr1,24650,-1,NaN,protein_coding,24651
58,chrKI270711chr.chr1,29626,-1,NaN,protein_coding,29627
628,chrKI270734chr.chr1,161852,-1,NaN,protein_coding,161853
629,chrKI270734chr.chr1,161750,-1,NaN,protein_coding,161751
713,chrKI270734chr.chr1,131494,1,NaN,protein_coding,131495
...,...,...,...,...,...,...
242597,chr1,154257071,1,UBAP2L,protein_coding,154257072
242604,chr1,154272589,1,HAX1,protein_coding,154272590
242605,chr1,154272609,1,HAX1,protein_coding,154272610
242607,chr1,154272629,1,HAX1,protein_coding,154272630


In [13]:
# save TSS_ann also to .bed

annotation["End"] = annotation["Start"] + 1
annotation["Score"] = None
annotation[["Chromosome", "Start", "End", "Gene", "Score", "Strand"]].to_csv(
    "tss.bed", sep="\t", index=False, header=None
)

In [6]:
fragments_list = sorted(glob.glob("../1_data_repository/fixedcells_fragments/*.tsv.gz"))
fragments_dict = {}
for fragments_file in fragments_list:
    sample = fragments_file.split("/")[-1].split(".fragments.tsv.gz")[0]
    fragments_dict[sample] = fragments_file
fragments_dict

{'BIO_ddseq_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_1.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_2.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_3.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_3.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_4.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_4.FIXEDCELLS.fragments.tsv.gz',
 'BRO_mtscatac_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BRO_mtscatac_1.FIXEDCELLS.fragments.tsv.gz',
 'BRO_mtscatac_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BRO_mtscatac_2.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xmultiome_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xmultiome_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xv11_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragmen

In [7]:
regions_paths_dict = {
    x.split("/")[-1].split(f"__")[0]: x
    for x in sorted(glob.glob("SCREEN_peaks/*consensus_peaks.bed"))
}
regions_paths_dict

{'BIO_ddseq_1.FIXEDCELLS': 'SCREEN_peaks/BIO_ddseq_1.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_2.FIXEDCELLS': 'SCREEN_peaks/BIO_ddseq_2.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_3.FIXEDCELLS': 'SCREEN_peaks/BIO_ddseq_3.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_4.FIXEDCELLS': 'SCREEN_peaks/BIO_ddseq_4.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'BRO_mtscatac_1.FIXEDCELLS': 'SCREEN_peaks/BRO_mtscatac_1.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'BRO_mtscatac_2.FIXEDCELLS': 'SCREEN_peaks/BRO_mtscatac_2.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'CNA_10xmultiome_1.FIXEDCELLS': 'SCREEN_peaks/CNA_10xmultiome_1.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'CNA_10xmultiome_2.FIXEDCELLS': 'SCREEN_peaks/CNA_10xmultiome_2.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'CNA_10xv11_1.FIXEDCELLS': 'SCREEN_peaks/CNA_10xv11_1.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'CNA_10xv11_2.FIXEDCELLS': 'SCREEN_peaks/CNA_10xv11_2.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'CNA_10xv11_3.FIXED

In [8]:
regions_paths_dict["VIB_hydrop_11.FIXEDCELLS"] = "SCREEN_peaks/VIB_hydrop_1.FIXEDCELLS__SCREEN_consensus_peaks.bed"
regions_paths_dict["VIB_hydrop_12.FIXEDCELLS"] = "SCREEN_peaks/VIB_hydrop_1.FIXEDCELLS__SCREEN_consensus_peaks.bed"
regions_paths_dict["VIB_hydrop_21.FIXEDCELLS"] = "SCREEN_peaks/VIB_hydrop_2.FIXEDCELLS__SCREEN_consensus_peaks.bed"
regions_paths_dict["VIB_hydrop_22.FIXEDCELLS"] = "SCREEN_peaks/VIB_hydrop_2.FIXEDCELLS__SCREEN_consensus_peaks.bed"

Now, make a sub dictionary of all samples within the fragments dict that have not been run yet (good for resuming a stopped cistopic run):

In [9]:
cistopic_qc_out = os.path.join(wdir, "cistopic_qc_out_CONSENSUS")
if not os.path.exists(cistopic_qc_out):
    os.makedirs(cistopic_qc_out)

In [10]:
fragments_sub_dict = {}
regions_sub_dict = {}
for sample in regions_paths_dict.keys():
    metadata_file = os.path.join(cistopic_qc_out, sample + "__metadata_bc.pkl")
    print(f"Checking if {metadata_file} exist...")
    if os.path.exists(metadata_file):
        print("\tMetadata exists! Skipping...")
    else:
        fragments_sub_dict[sample] = fragments_dict[sample]
        print("\tMetadata does not exist, adding to subdict to generate")

Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_3_cistopic_consensus/cistopic_qc_out_CONSENSUS/BIO_ddseq_1.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata exists! Skipping...
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_3_cistopic_consensus/cistopic_qc_out_CONSENSUS/BIO_ddseq_2.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata exists! Skipping...
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_3_cistopic_consensus/cistopic_qc_out_CONSENSUS/BIO_ddseq_3.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata exists! Skipping...
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_3_cistopic_consensus/cistopic_qc_out_CONSENSUS/BIO_ddseq_4.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata exists! Skipping...
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_3_cistopic_consensus/cistopic_q

In [11]:
regions_sub_dict = {x: regions_paths_dict[x] for x in sorted(fragments_sub_dict.keys())}
regions_sub_dict

{'VIB_hydrop_11.FIXEDCELLS': 'SCREEN_peaks/VIB_hydrop_1.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'VIB_hydrop_12.FIXEDCELLS': 'SCREEN_peaks/VIB_hydrop_1.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'VIB_hydrop_21.FIXEDCELLS': 'SCREEN_peaks/VIB_hydrop_2.FIXEDCELLS__SCREEN_consensus_peaks.bed',
 'VIB_hydrop_22.FIXEDCELLS': 'SCREEN_peaks/VIB_hydrop_2.FIXEDCELLS__SCREEN_consensus_peaks.bed'}

In [12]:
ray.shutdown()

In [13]:
n_cores = 8
if regions_sub_dict != {}:
    samples_sub = list(regions_sub_dict.keys())
    blocks = [samples_sub[i : i + n_cores] for i in range(0, len(samples_sub), n_cores)]
    for samples_torun_in_block in blocks:
        fragments_sub_dict_block = {
            key: fragments_sub_dict[key] for key in samples_torun_in_block
        }
        regions_sub_dict_block = {
            key: regions_sub_dict[key] for key in samples_torun_in_block
        }

        metadata_bc_dict, profile_data_dict = compute_qc_stats(
            fragments_dict=fragments_sub_dict_block,
            tss_annotation=annotation,
            stats=[
                "barcode_rank_plot",
                "duplicate_rate",
                "insert_size_distribution",
                "profile_tss",
                "frip",
            ],
            label_list=None,
            path_to_regions=regions_sub_dict_block,
            n_cpu=n_cores,
            valid_bc=None,
            n_frag=10,
            n_bc=None,
            tss_flank_window=2000,
            tss_window=50,
            tss_minimum_signal_window=100,
            tss_rolling_window=10,
            # min_norm=0.2,
            remove_duplicates=True,
        )

        ray.shutdown()
        print(f"Dumping files in {cistopic_qc_out}...")
        for sample in sorted(metadata_bc_dict.keys()):
            metadata_bc_dict[sample]["sample_id"] = sample
            metadata_bc_dict[sample].index = [
                x + "___" + sample for x in list(metadata_bc_dict[sample].index)
            ]
            with open(
                os.path.join(cistopic_qc_out, f"{sample}__metadata_bc.pkl"), "wb"
            ) as f:
                pickle.dump(metadata_bc_dict[sample], f, protocol=4)

            with open(
                os.path.join(cistopic_qc_out, f"{sample}__profile_data.pkl"), "wb"
            ) as f:
                pickle.dump(profile_data_dict[sample], f, protocol=4)
else:
    print("All samples already processed.")

2022-09-16 16:02:24,220 cisTopic     INFO     n_cpu is larger than the number of samples. Setting n_cpu to the number of samples


2022-09-16 16:02:28,277	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265


(compute_qc_stats_ray pid=3490945) 2022-09-16 16:02:42,302 cisTopic     INFO     Reading VIB_hydrop_12.FIXEDCELLS
(compute_qc_stats_ray pid=3490948) 2022-09-16 16:02:42,305 cisTopic     INFO     Reading VIB_hydrop_11.FIXEDCELLS
(compute_qc_stats_ray pid=3490947) 2022-09-16 16:02:42,360 cisTopic     INFO     Reading VIB_hydrop_21.FIXEDCELLS
(compute_qc_stats_ray pid=3490946) 2022-09-16 16:02:42,362 cisTopic     INFO     Reading VIB_hydrop_22.FIXEDCELLS
(compute_qc_stats_ray pid=3490946) 2022-09-16 16:03:21,297 cisTopic     INFO     Computing barcode rank plot for VIB_hydrop_22.FIXEDCELLS
(compute_qc_stats_ray pid=3490946) 2022-09-16 16:03:21,297 cisTopic     INFO     Counting fragments
(compute_qc_stats_ray pid=3490946) 2022-09-16 16:03:23,680 cisTopic     INFO     Marking barcodes with more than 10
(compute_qc_stats_ray pid=3490946) 2022-09-16 16:03:23,786 cisTopic     INFO     Returning plot data
(compute_qc_stats_ray pid=3490946) 2022-09-16 16:03:23,808 cisTopic     INFO     Returnin

# Plot

Calculating a KDE is simultaneously expensive and scales poorly with increasing n. Therefore, I wrote a multithreaded script that divides the QC array into equal parts (interleaved to avoid biases in the order!) and performs a KDE calculation on each part. Here, Otsu thresholding is used to find the right threshold for minimum fragments and minimum TSS enrichment. ddseq samples have a significantly higher noise floor than the other samples when it comes to fragment distribution. Therefore, the otsu algorithm is performed on all barcodes with a minimum of 300 fragments for the ddseq samples, and a minimum of 100 fragments for all the other samples. I tried to perform this filtering completely independent of sample/technique (e.g. using gaussian mixture modeling, Jenks natural breaks, or multiple step Otsu thresholding) but found that no solution worked perfectly for all samples.

This is regulated by the code below in qc_plots.py:
```
min_otsu_frags_dict = {}
for fragments_file in fragments_list:
    sample = fragments_file.split("/")[-1].split(".")[0]
    tech = sample.split('_')[1]
    if tech == "ddseq":
        if sample == "BIO_ddseq_1":
            min_otsu_frags_dict[sample] = 600
        else:
            min_otsu_frags_dict[sample] = 300
    elif tech == "hydrop":
        min_otsu_frags_dict[sample] = 300
    else:
        min_otsu_frags_dict[sample] = 100
```

In [14]:
!cat ../0_resources/scripts/qc_plots.py

import kde
import pycisTopic
import glob
import os
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import multiprocess as mp
from multiprocess import Pool
import pprint as pp

def histogram(array, nbins=100):
    """
    Draw histogram from distribution and identify centers.
    Parameters
    ---------
    array: `class::np.array`
            Scores distribution
    nbins: int
            Number of bins to use in the histogram
    Return
    ---------
    float
            Histogram values and bin centers.
    """
    array = array.ravel().flatten()
    hist, bin_edges = np.histogram(array, bins=nbins, range=None)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2.0
    return hist, bin_centers


def threshold_otsu(array, nbins=100, min_value=100):
    """
    Apply Otsu threshold on topic-region distributions [Otsu, 1979].
    Parameters
    ---------
    array: `class::np.array`
            Array containing the region va

Since multiprocessing does not work with jupyter notebooks, run the following code in terminal:

```
mkdir plots_qc
mkdir selected_barcodes
SIF=../0_resources/cistopic_image/20220722_pycistopic.sif
singularity exec \
    --cleanenv \
    -H $PWD:/home \
    $SIF \
    python ../0_resources/scripts/qc_plots.py
```

And then open the plots:

In [15]:
metadata_bc_pkl_list = sorted(glob.glob("cistopic_qc_out/*metadata_bc.pkl"))
metadata_bc_pkl_path_dict = {}
for metadata_bc_pkl_path in metadata_bc_pkl_list:
    sample = metadata_bc_pkl_path.split("/")[-1].split("__")[0]
    metadata_bc_pkl_path_dict[sample] = metadata_bc_pkl_path

for sample in metadata_bc_pkl_path_dict.keys():
    if os.path.exists(f"selected_barcodes/{sample}_bc_passing_filters_otsu.pkl"):
        print(f"{sample} bc passing filters exists, printing img and skipping")
        display(Image(f"plots_qc/{sample}_qc_otsu.png"))
    else:
        print(
            f"{sample} bc passing filters does not exist yet, generate using qc_plots.py script!"
        )

# Write raw barcode file

Freemuxlet reads barcodes in the bam file. These barcodes (tag: DB) do not contain the `sample` suffix, which is included in the `*_bc_passing_filters_otsu.txt` files:

In [16]:
!head selected_barcodes/BIO_ddseq_1.FIXEDCELLS_bc_passing_filters_otsu.txt

AAGGTTCTTTCCTAAGAGACC_CCGCGATCACGCCAATTATTC_CTGCGCCTGTTTAGATTGCCT___BIO_ddseq_1.FULL
GAGAGGTGACAATATGGTGAA___BIO_ddseq_1.FULL
CGATTACGCTCCAAGAGCTAA_CTAGAGCACACGCGTACTTTC_GGCGTCCTAACGCCCGTAGCC___BIO_ddseq_1.FULL
CTAGAGCGGTTAGTGTGGCGC_GCTTTATTTGGACTCGAGGTC_GTTGATGTCATACAGTGGTAG___BIO_ddseq_1.FULL
CGCGCTTTCTTGTAAGCCGCC_CTTGTCCCGTGATATGATTGT___BIO_ddseq_1.FULL
AGCCGCCGCACAGCCCATAAT_CGAGTGGTGGTGAATAGTGTT_GAATCAACGAGTGGCAGGCGG___BIO_ddseq_1.FULL
CACATGATGCGAGCCTCATTT_CAGAGAGTGGTGAAACCTACC_GAATATGTGACCGCTGGTGAA_TCAAGACCGGATCTATTGCCT___BIO_ddseq_1.FULL
CGAGTGGGAGGACAAGATAGG_CGTGATAATGAATATGCGAGC_TCAAGACGTGTGTCAACTCTT___BIO_ddseq_1.FULL
CTCTTGCTGCGTTGCGGCTGC_TCGACAGCGGTACGAACGTAA___BIO_ddseq_1.FULL
CCACTTGCACATGACTAACTC_TATGTGCACGAGAACCAAGCT___BIO_ddseq_1.FULL


In [17]:
for file in glob.glob("selected_barcodes/*_bc_passing_filters_otsu.txt"):
    print(file)
    sample = file.split("/")[-1].split("_bc_passing_filters_otsu.txt")[0]
    print(sample)
    df = pd.read_csv(file, header=None, index_col=None)

    with open(f"selected_barcodes/{sample}_bc_passing_filters_otsu.RAW.txt", "w") as fp:
        for x in df[0]:
            # write each item on a new line
            fp.write(x.split("___")[0] + "\n")
        print("Done")

selected_barcodes/MDC_mtscatac_1.FIXEDCELLS_bc_passing_filters_otsu.txt
MDC_mtscatac_1.FIXEDCELLS
Done
selected_barcodes/STA_10xv11_2.FIXEDCELLS_bc_passing_filters_otsu.txt
STA_10xv11_2.FIXEDCELLS
Done
selected_barcodes/CNA_10xv11_1.FIXEDCELLS_bc_passing_filters_otsu.txt
CNA_10xv11_1.FIXEDCELLS
Done
selected_barcodes/CNA_10xmultiome_1.FIXEDCELLS_bc_passing_filters_otsu.txt
CNA_10xmultiome_1.FIXEDCELLS
Done
selected_barcodes/SAN_10xmultiome_1.FIXEDCELLS_bc_passing_filters_otsu.txt
SAN_10xmultiome_1.FIXEDCELLS
Done
selected_barcodes/VIB_10xmultiome_2.FIXEDCELLS_bc_passing_filters_otsu.txt
VIB_10xmultiome_2.FIXEDCELLS
Done
selected_barcodes/STA_10xv11_1.FIXEDCELLS_bc_passing_filters_otsu.txt
STA_10xv11_1.FIXEDCELLS
Done
selected_barcodes/VIB_hydrop_11.FIXEDCELLS_bc_passing_filters_otsu.txt
VIB_hydrop_11.FIXEDCELLS
Done
selected_barcodes/MDC_mtscatac_2.FIXEDCELLS_bc_passing_filters_otsu.txt
MDC_mtscatac_2.FIXEDCELLS
Done
selected_barcodes/EPF_hydrop_3.FIXEDCELLS_bc_passing_filters_otsu.txt

In [18]:
!head selected_barcodes/BIO_ddseq_1.FIXEDCELLS_bc_passing_filters_otsu.RAW.txt

AAGGTTCTTTCCTAAGAGACC_CCGCGATCACGCCAATTATTC_CTGCGCCTGTTTAGATTGCCT
GAGAGGTGACAATATGGTGAA
CGATTACGCTCCAAGAGCTAA_CTAGAGCACACGCGTACTTTC_GGCGTCCTAACGCCCGTAGCC
CTAGAGCGGTTAGTGTGGCGC_GCTTTATTTGGACTCGAGGTC_GTTGATGTCATACAGTGGTAG
CGCGCTTTCTTGTAAGCCGCC_CTTGTCCCGTGATATGATTGT
AGCCGCCGCACAGCCCATAAT_CGAGTGGTGGTGAATAGTGTT_GAATCAACGAGTGGCAGGCGG
CACATGATGCGAGCCTCATTT_CAGAGAGTGGTGAAACCTACC_GAATATGTGACCGCTGGTGAA_TCAAGACCGGATCTATTGCCT
CGAGTGGGAGGACAAGATAGG_CGTGATAATGAATATGCGAGC_TCAAGACGTGTGTCAACTCTT
CTCTTGCTGCGTTGCGGCTGC_TCGACAGCGGTACGAACGTAA
CCACTTGCACATGACTAACTC_TATGTGCACGAGAACCAAGCT
